### 핵심 아이디어

분류 작업을 두 번 하자.
그래도 미분류된 얘들은 유사도 분석을 통해 분류하자.

### 결과
두번 분류하는 것은 좋은 성과를 얻음.
유사도 분석을 통한 분류는 좋지 못함. 여기부턴 수작업의 영역이 아닐까 생각됨. 

In [1]:
# !pip install gensim --upgrade
# !pip install --upgrade numpy

In [2]:
# !pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import gensim
from konlpy.tag import Okt
okt = Okt()

In [4]:
import pandas as pd
import numpy as np
import re

In [5]:
# 데이터 불러오기
# 표준데이터셋으로 연습합니다
df = pd.read_csv('/content/bfe1403f648f11f9.csv')

In [6]:
# 한글 외 제거하고 메뉴를 리스트로 호출 
menu = df['title'].apply(lambda x: re.sub('[^0-9가-힣]', '', x)).tolist()

In [7]:
# 메뉴리스트트를 okt 토크나이저로 토큰화화
menu_token = [okt.morphs(i) for i in menu]

In [8]:
# 토큰화한 메뉴명에다가 메뉴명을 또 집어넣음음
for i, v in enumerate(menu):
  menu_token[i].append(v)

In [9]:
# 마치 이렇게!
menu_token[:10]

[['소', '다', '에', '이드', '소다에이드'],
 ['블루베리', '요거트', '스무디', '블루베리요거트스무디'],
 ['플레인', '요거트', '스무디', '플레인요거트스무디'],
 ['소', '다', '스무디', '소다스무디'],
 ['블루베리', '스무디', '블루베리스무디'],
 ['청포도', '스무디', '청포도스무디'],
 ['살구', '스무디', '살구스무디'],
 ['망고', '스무디', '망고스무디'],
 ['자두', '스무디', '자두스무디'],
 ['딸기', '바나나', '스무디', '딸기바나나스무디']]

In [10]:
# 워드 투 벡으로 벡터화를 시도하는 중...
from gensim.models.word2vec import Word2Vec
model = Word2Vec(sentences=menu_token, size=100, window=5, min_count=1, workers=4, batch_words=50,iter=200)

In [11]:
# 워드투벡 유사도 측정 샘플플
model.wv.similarity('치즈돈까스유부초밥', '돈까스')

0.9866194

In [12]:
# 토큰화한 메뉴 리스트의 리스트를 다 풀어버림
menu_token_hasing = [j for i in menu_token for j in i if j != '']
menu_token_hasing[:10]

['소', '다', '에', '이드', '소다에이드', '블루베리', '요거트', '스무디', '블루베리요거트스무디', '플레인']

In [13]:
# 그리고 토큰의 빈도를 카운팅함함
menu_count = {}

for i in menu_token_hasing:
  if i not in menu_count:
    menu_count[i] = 1
  else:
    menu_count[i] += 1

In [14]:
# 카운팅된 토큰을 빈도내림차순으로 정렬렬
items = menu_count.items()
sorted_items = sorted(items, key=lambda x: x[1], reverse=True) # 빈도 내림차순 정렬
sorted_items[:10]

[('라떼', 66),
 ('크림', 55),
 ('초코', 37),
 ('에', 36),
 ('도넛', 36),
 ('스무디', 34),
 ('티', 34),
 ('이드', 32),
 ('딸기', 32),
 ('치즈', 30)]

In [15]:
# 1글자 토큰을 모두 제거하고, 특정 빈도 이상의 토큰만 남김
sorted_items_rank = [v for v, c in sorted_items if c >= 4 and len(v) != 1]

# 걸러진 토큰을 길이 내림차순으로 정렬
sorted_items_rank = sorted(sorted_items_rank, key=lambda x: len(x), reverse=True)
sorted_items_rank[:10]

['플레인요거트스무디',
 '딸기요거트스무디',
 '망고요거트스무디',
 '청포도에이드',
 '쿠앤크프라페',
 '아메리카노',
 '레몬에이드',
 '치즈케이크',
 '자두에이드',
 '바닐라라떼']

In [16]:
# 만약 아래 go_words 토큰의 빈도가 일정 수 이상이면, 토큰 목록에 추가함 

go_words = ['빵', '잼', '티', '팥', '떡', '꿀','밥', '탕', '면']

for i in go_words:
  if menu_count[i] >= 4:
    sorted_items_rank.append(i)

In [17]:
# 특정 토큰으로 종결되는 메뉴명을 그 토큰으로 분류함함

from tqdm import tqdm

menu_cluster = {}

for food_token in tqdm(sorted_items_rank):
  for food_name in menu:
    if food_name not in menu_cluster: # 이미 분류된 메뉴는 건들지 않음

      # 메뉴명이 토큰명보다 길면서 동시에 토큰명으로 끝나는 메뉴를 토큰 카테고리로 분류 
      if len(food_token) <= len(food_name) and food_name[-len(food_token):] == food_token:
        menu_cluster[food_name] = food_token

100%|██████████| 126/126 [00:00<00:00, 2014.82it/s]


In [18]:
# 분류된 결과 
menu_cluster

{'플레인요거트스무디': '플레인요거트스무디',
 '딸기요거트스무디': '딸기요거트스무디',
 '망고요거트스무디': '망고요거트스무디',
 '청포도에이드': '청포도에이드',
 '레몬청포도에이드': '청포도에이드',
 '쿠앤크프라페': '쿠앤크프라페',
 '헤이즐넛아메리카노': '아메리카노',
 '아메리카노': '아메리카노',
 '스폐셜아메리카노': '아메리카노',
 '디카페인아메리카노': '아메리카노',
 '스페셜티아메리카노': '아메리카노',
 '타임세일아메리카노': '아메리카노',
 '아샷추아이스티아메리카노': '아메리카노',
 '오픈기념아메리카노': '아메리카노',
 '행사아메리카노': '아메리카노',
 '치즈크림아메리카노': '아메리카노',
 '꿀아메리카노': '아메리카노',
 '레몬에이드': '레몬에이드',
 '핑크레몬에이드': '레몬에이드',
 '블루레몬에이드': '레몬에이드',
 '레몬치즈케이크': '치즈케이크',
 '까망베르치즈케이크': '치즈케이크',
 '초코쿠키치즈케이크': '치즈케이크',
 '시카고치즈케이크': '치즈케이크',
 '당근크림치즈케이크': '치즈케이크',
 '바스크치즈케이크': '치즈케이크',
 '자두에이드': '자두에이드',
 '바닐라라떼': '바닐라라떼',
 '오트밀바닐라라떼': '바닐라라떼',
 '블루베리': '블루베리',
 '머핀초코블루베리': '블루베리',
 '카페라떼': '카페라떼',
 '오트밀카페라떼': '카페라떼',
 '사이즈업아이스카페라떼': '카페라떼',
 '치즈크림카페라떼': '카페라떼',
 '꿀카페라떼': '카페라떼',
 '생크림카스테라': '카스테라',
 '우유크림카스테라': '카스테라',
 '크림카스테라': '카스테라',
 '복숭아아이스티': '아이스티',
 '복숭아이스티': '아이스티',
 '레드향라봉아이스티': '아이스티',
 '리얼복숭아아이스티': '아이스티',
 '리얼망고아이스티': '아이스티',
 '리얼청포도아이스티': '아이스티',
 '아이스티': '아이스티',
 '홍차얼그레이': '얼그레이

In [19]:
# 분류된 결과를 sample이라는 데이터프레임으로 저장했음 
sample = pd.DataFrame(columns=['title', 'category'], data=menu_cluster.items())
sample

,title,category
0,플레인요거트스무디,플레인요거트스무디
1,딸기요거트스무디,딸기요거트스무디
2,망고요거트스무디,망고요거트스무디
3,청포도에이드,청포도에이드
4,레몬청포도에이드,청포도에이드
...,...,...
419,로얄프리미엄밀크티,티
420,핑크선셋베리티,티
421,옐로우캐모마일티,티
422,밀크티,티


In [20]:
# 미분류 메뉴 확인
non_category = []
for i in menu:
  if i not in menu_cluster:
    non_category.append(i)

len(menu), len(non_category)

(775, 270)

In [21]:
# 미분류된 메뉴들 
non_category[:10]

['한라봉차',
 '유자차',
 '죠리퐁프라페',
 '딸기초코프라페',
 '초바퐁프라페초코바나나죠리퐁',
 '쑥프라페',
 '민트초코프라페',
 '녹차프라페',
 '더블초코프라페',
 '돼지바프라페']

In [22]:
# n-gram 구현 함수 간단하게
def to_ngrams(words, n):
  ngrams = []
  for b in range(0, len(words) - n + 1):
      ngrams.append(words[b:b+n])
  return ngrams

In [23]:
# 미분류된 메뉴들을 또 다시 토큰화하고 
non_category_token = [okt.morphs(i) for i in non_category]

In [24]:
# 거기에 n-gram으로 짤라서 또 더해줌. 즉 okt토크나이저로 토큰화 + n-gram으로 토큰화 
non_category_token += [to_ngrams(i, 2) for i in non_category]
non_category_token += [to_ngrams(i, 3) for i in non_category]
non_category_token_hasing = [j for i in non_category_token for j in i]
non_category_token_hasing[:10]

['한라봉', '차', '유자차', '죠리퐁', '프라', '페', '딸기', '초코', '프라', '페']

In [25]:
# 다시 토큰의 빈도를 측정함 
menu_count2 = {}

for i in non_category_token_hasing:
  if i not in menu_count2:
    menu_count2[i] = 1
  else:
    menu_count2[i] += 1

In [44]:
# 빈도 측정 결과 
menu_count2

{'한라봉': 2,
 '차': 4,
 '유자차': 4,
 '죠리퐁': 4,
 '프라': 30,
 '페': 14,
 '딸기': 10,
 '초코': 20,
 '초바': 2,
 '퐁프라페': 1,
 '초코바나나': 1,
 '쑥': 1,
 '민트': 6,
 '녹차': 4,
 '더블': 4,
 '돼지': 2,
 '바프': 2,
 '라': 3,
 '밀크': 4,
 '티': 2,
 '바닐라': 11,
 '청귤차': 2,
 '체리': 4,
 '콕': 1,
 '자몽': 2,
 '카라멜': 6,
 '마끼아또': 2,
 '레몬': 8,
 '가유': 4,
 '크로': 33,
 '플': 16,
 '미숫가루': 1,
 '쿠키': 27,
 '슈': 16,
 '츄러스': 2,
 '무화과': 2,
 '얼그레이': 1,
 '쇼콜라': 4,
 '진한': 2,
 '원두': 4,
 '레아': 2,
 '치즈': 26,
 '타르트': 5,
 '갸토': 2,
 '에그타르트': 1,
 '스모': 4,
 '어': 3,
 '쿠앤': 3,
 '크': 1,
 '휘': 1,
 '낭시': 2,
 '에': 1,
 '티라미수': 2,
 '르완다': 2,
 '마': 2,
 '쿠바': 2,
 '풀리': 2,
 '콜드': 6,
 '폼': 1,
 '노체': 2,
 '브루': 4,
 '디카': 4,
 '페인': 4,
 '콜롬비아': 2,
 '에디오피아': 1,
 '아이스크림': 3,
 '라테': 2,
 '캐러멜마키아토': 1,
 '아인': 2,
 '슈페': 4,
 '너': 2,
 '에티오피아': 3,
 '모모': 2,
 '구지': 2,
 '브라질': 2,
 '만치': 2,
 '케이': 3,
 '라드': 2,
 '미나스': 2,
 '워': 1,
 '시드': 2,
 '내추럴': 2,
 '코스타리카': 1,
 '과테말라': 2,
 '스': 2,
 '위트': 4,
 '카모마일': 2,
 '라즈베리': 2,
 '피치': 2,
 '세리토스': 1,
 '휘핑있음': 1,
 '휘핑없음': 1,
 '선택': 4,
 '없음': 5,
 '조금':

In [27]:
# 똑같이 빈도 내림차순으로 정렬하고 
items2 = menu_count2.items()
sorted_items2 = sorted(items2, key=lambda x: x[1], reverse=True) # 빈도 내림차순 정렬

In [28]:
# 정렬된 토큰 
sorted_items2

[('크로', 33),
 ('프라', 30),
 ('쿠키', 27),
 ('치즈', 26),
 ('초코', 20),
 ('플', 16),
 ('슈', 16),
 ('라페', 16),
 ('로플', 16),
 ('프라페', 16),
 ('크로플', 16),
 ('페', 14),
 ('리얼', 14),
 ('키슈', 14),
 ('쿠키슈', 14),
 ('크림', 13),
 ('바닐라', 11),
 ('딸기', 10),
 ('스무디', 10),
 ('포케', 10),
 ('베리', 10),
 ('커스', 9),
 ('레몬', 8),
 ('우유', 8),
 ('시나몬', 8),
 ('커스터드', 8),
 ('스터', 8),
 ('터드', 8),
 ('커스터', 8),
 ('스터드', 8),
 ('배배', 7),
 ('콜라', 7),
 ('드쿠', 7),
 ('터드쿠', 7),
 ('드쿠키', 7),
 ('민트', 6),
 ('카라멜', 6),
 ('콜드', 6),
 ('샐러드', 6),
 ('파이', 6),
 ('페스', 6),
 ('츄리', 6),
 ('코코', 6),
 ('빙수', 6),
 ('치노', 6),
 ('변경', 6),
 ('꾸꼬', 6),
 ('알콜', 6),
 ('맥주', 6),
 ('어묵', 6),
 ('포장', 6),
 ('할인', 6),
 ('우동', 6),
 ('연어', 6),
 ('롤', 6),
 ('바닐', 6),
 ('닐라', 6),
 ('타르트', 5),
 ('없음', 5),
 ('블랙', 5),
 ('아이스', 5),
 ('페너', 5),
 ('배배꼬', 5),
 ('볼', 5),
 ('장어', 5),
 ('바나나', 5),
 ('사이다', 5),
 ('카츠', 5),
 ('유크', 5),
 ('스트', 5),
 ('스무', 5),
 ('무디', 5),
 ('차', 4),
 ('유자차', 4),
 ('죠리퐁', 4),
 ('녹차', 4),
 ('더블', 4),
 ('밀크', 4),
 ('체리', 4),
 ('가유', 4),
 ('쇼

In [29]:
# 첫번째 분류와 같은 절차로 토큰 정렬 
go_words = ['티','꿀', '탕', '면', '차']
sorted_items_rank2 = [v for v, c in sorted_items2 if c >= 10 if len(v) != 1 or v in go_words]
sorted_items_rank2 = sorted(sorted_items_rank2, key=lambda x: len(x), reverse=True)

for i in go_words:
  if menu_count2[i] >= 10:
    sorted_items_rank.append(i)

sorted_items_rank2

['프라페',
 '크로플',
 '쿠키슈',
 '바닐라',
 '스무디',
 '크로',
 '프라',
 '쿠키',
 '치즈',
 '초코',
 '라페',
 '로플',
 '리얼',
 '키슈',
 '크림',
 '딸기',
 '포케',
 '베리']

In [30]:
# 미분류된 메뉴명 => 거기서 다시 추출한 토큰 + n-gram 토큰 => 다시 분류
# 즉 분류작업을 두 번 진행했음

menu_cluster2 = {}

for food_token in tqdm(sorted_items_rank2):
  for food_name in non_category:
    if food_name not in menu_cluster2: # 이미 분류된 메뉴는 건들지 않음

      # 메뉴명이 토큰명보다 길면서 동시에 토큰명으로 끝나는 메뉴를 토큰 카테고리로 분류 
      if len(food_token) <= len(food_name) and food_name[-len(food_token):] == food_token:
        menu_cluster2[food_name] = food_token

100%|██████████| 18/18 [00:00<00:00, 3256.45it/s]


In [31]:
# 분류된 결과 
menu_cluster2

{'죠리퐁프라페': '프라페',
 '딸기초코프라페': '프라페',
 '쑥프라페': '프라페',
 '민트초코프라페': '프라페',
 '녹차프라페': '프라페',
 '더블초코프라페': '프라페',
 '돼지바프라페': '프라페',
 '밀크티프라페': '프라페',
 '바닐라프라페': '프라페',
 '자바칩프라페': '프라페',
 '민트초코칩프라페': '프라페',
 '헤이즐넛프라페': '프라페',
 '커피프라페': '프라페',
 '이천쌀프라페': '프라페',
 '가유크로플': '크로플',
 '브라운치즈크로플': '크로플',
 '아이스크림크로플': '크로플',
 '시나몬크로플': '크로플',
 '인절미크로플': '크로플',
 '누텔라크로플': '크로플',
 '체다치즈크로플': '크로플',
 '크림치즈크로플': '크로플',
 '애플시나몬크로플': '크로플',
 '딸기크로플': '크로플',
 '블루베리크로플': '크로플',
 '초코크런치크로플': '크로플',
 '티라미수크로플': '크로플',
 '콘체다크로플': '크로플',
 '플레인크로플': '크로플',
 '쿠키슈': '쿠키슈',
 '라즈베리커스터드쿠키슈': '쿠키슈',
 '딥초코커스터드쿠키슈': '쿠키슈',
 '바닐라커스터드쿠키슈': '쿠키슈',
 '우유쿠키슈': '쿠키슈',
 '리얼밀크카라멜쿠키슈': '쿠키슈',
 '오레오순우유크림쿠키슈': '쿠키슈',
 '리얼블루베리쿠키슈': '쿠키슈',
 '리얼커스터드쿠키슈': '쿠키슈',
 '리얼치즈커스터드쿠키슈': '쿠키슈',
 '리얼스트로베리쿠키슈': '쿠키슈',
 '리얼말차커스터드쿠키슈': '쿠키슈',
 '리얼초코커스터드쿠키슈': '쿠키슈',
 '딥초코쿠키슈': '쿠키슈',
 '장어포케': '포케',
 '메밀면포케': '포케',
 '현미밥포케': '포케',
 '채소만포케': '포케',
 '라즈베리': '베리',
 '퍼플스무디볼바나나망고블루베리블랙베리': '베리'}

In [46]:
# 두번째 분류된 메뉴명을 데이터프레임화 했음 
sample2 = pd.DataFrame(columns=['title', 'category'], data=menu_cluster2.items())

# 첫번째 분류된 메뉴명 데이터프레임과 합치기 
sample_df = pd.concat([sample, sample2])
sample_df.sample(30)

,title,category
312,오징어튀김,튀김
39,리얼스트로베리쿠키슈,쿠키슈
316,야끼만두튀김,튀김
328,망고주스,주스
249,샷추가,추가
252,소세지추가,추가
403,패션후르츠얼그레이티,티
181,제주말차라떼,라떼
118,닭갈비가래떡볶이,떡볶이
173,생딸기라떼,라떼


In [33]:
# 두번 분류했음에도 미분류된 얘들 확인 
non_category2 = []
for i in non_category:
  if i not in menu_cluster2:
    non_category2.append(i)

len(df), len(non_category2)

(775, 215)

In [34]:
# 아직도 남은 171개의 미분류는, 지금까지 생성된 분류토큰과 코사인 유사도를 측정해서 강제 분류할 예정

# 일단 지금까지 생성된 분류토큰 목록 가져오기
category_list = sample_df['category'].tolist()

# 중복 제거
category_list = list(set(category_list))

In [35]:
# 길이 내림차순 정렬렬
category_list = sorted(category_list, key=lambda x: len(x), reverse=True)

In [36]:
# 각 글자 단위로 유사도를 측정하기 위해, 글자 단위로 토크나이징을 진행
category_list_part_away = [' '.join(i) for i in category_list]
category_list_part_away[:10]

['플 레 인 요 거 트 스 무 디',
 '딸 기 요 거 트 스 무 디',
 '망 고 요 거 트 스 무 디',
 '쿠 앤 크 프 라 페',
 '청 포 도 에 이 드',
 '치 즈 케 이 크',
 '자 두 에 이 드',
 '아 메 리 카 노',
 '바 닐 라 라 떼',
 '레 몬 에 이 드']

In [37]:
# 아직까지 미분류된 메뉴명들도 글자 단위로 토크나이징 진행
non_category2_part_away = [' '.join(i) for i in non_category2]
non_category2_part_away[:10]

['한 라 봉 차',
 '유 자 차',
 '초 바 퐁 프 라 페 초 코 바 나 나 죠 리 퐁',
 '청 귤 차',
 '체 리 콕',
 '자 몽 차',
 '카 라 멜 마 끼 아 또',
 '레 몬 차',
 '미 숫 가 루',
 '츄 러 스']

In [38]:
# 글자끼리 비교해서 유사도를 측정하기 위한 작업들 (원핫벡터 매핑기)
from sklearn.preprocessing import MultiLabelBinarizer
lb = MultiLabelBinarizer()
vocab = non_category2_part_away + category_list_part_away
vocab = list(set([j for i in vocab for j in i]))

# 생성된 단어사전
vocab[:10]

['콜', '많', '일', '지', '뎅', '금', '에', '콤', '가', '고']

In [39]:
# 유사도 측정을 위한 학습...
lb.fit(vocab)

MultiLabelBinarizer()

In [40]:
# 예시
sim1 = lb.transform(['빨노핫'])
sim2 = lb.transform(['핫이주쿨'])

# 생성된 원핫벡터 
sim1, sim2

(array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0

In [41]:
# 세번째 분류작업
from sklearn.metrics.pairwise import cosine_similarity

menu_cluster3 = {}
for food_token in category_list:
  for food_name in non_category2:
    if food_token not in menu_cluster3:

      # 메뉴명과 분류토큰을 원핫 벡터로 변환
      sim1 = lb.transform([food_token])
      sim2 = lb.transform([food_name])

      # 두 원핫 벡터의 코사인 유사도를 측정
      sim = cosine_similarity(sim1, sim2)

      # 측정된 유사도가 0.5 이상이면 메뉴명을 분류토큰으로 분류
      if sim >= 0.5:
        menu_cluster3[food_name] = food_token

In [42]:
# 결과: 똥망. 이쯤이면 수작업이 필요한듯
menu_cluster3

{'스무디볼': '스무디',
 '큐브망고스무디계절': '스무디',
 '쿠앤크스모어': '쿠앤크프라페',
 '쿠앤크코코팝': '쿠앤크프라페',
 '크림치즈프레즐': '치즈',
 '치즈볼': '치즈',
 '카프리': '아메리카노',
 '바닐라라떼아이스': '라떼',
 '바닐라코코팝': '바닐라',
 '바닐라웨하스': '바닐라',
 '바닐라비스킷슈': '바닐라',
 '레몬차': '레몬',
 '레몬맛': '레몬',
 '아이스크림라테': '테라',
 '아이스크림브라우니': '아이스티',
 '과테말라': '테라',
 '과테말라세리토스': '카스테라',
 '트로피카나스파클링오랜지': '스파클링',
 '블루베리파이': '베리',
 '모카번': '카페모카',
 '민트모카': '민트',
 '연어크림치즈아보카도롤': '아보카도',
 '치킨카츠아보카도롤': '아보카도',
 '연어아보카도롤': '연어',
 '치킨카츠': '치킨',
 '치즈카츠': '치즈',
 '티라미수타르트': '티라미수',
 '얼그레이쇼콜라': '얼그레이',
 '스위트카모마일': '카스',
 '크로크무슈': '크로플',
 '큐브수박스무디계절': '스무디',
 '허니버터베이글칩': '베이글',
 '갈릭버터베이글칩': '베이글',
 '스프라이트': '프라페',
 '말렌카케이크월넛코코아': '케이크',
 '플레인크러핀': '플레인',
 '핫도그프로틴어니언클래식': '핫도그',
 '애플시나몬파이': '애플',
 '시나몬바게트볼': '시나몬',
 '초바퐁프라페초코바나나죠리퐁': '프라페',
 '우유크러핀': '우유',
 '우유크로와상': '우유',
 '우동쯔유': '우동',
 '소시지페스츄리': '소스',
 '꿀레몬그라스차': '레몬',
 '메인토핑추가금': '추가',
 '레아치즈타르트': '치즈',
 '배배꼬치즈슈페너': '치즈',
 '치즈사리': '치즈',
 '치즈스틱': '치즈',
 '스트로베리파이': '베리',
 '라떼로변경': '라떼',
 '아인슈페너': '페인',
 '스콘건포도호두': '스콘',
 '연어스테이크롤